SETUP:
1. Download input_images from our drive folder and upload them to colab's runtime files

In [ ]:
import torch
import os
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
# setup (visualization library only supports CPU)
device = torch.device('cpu')

In [ ]:
# CNN Visualizer code: (https://github.com/utkuozbulak/pytorch-cnn-visualizations)


In [ ]:
# misc_functions.py

"""
Created on Thu Oct 21 11:09:09 2017
@author: Utku Ozbulak - github.com/utkuozbulak
"""
import os
import copy
import numpy as np
from PIL import Image
import matplotlib.cm as mpl_color_map
from matplotlib.colors import ListedColormap
from matplotlib import pyplot as plt

import torch
from torch.autograd import Variable
from torchvision import models


def convert_to_grayscale(im_as_arr):
    """
        Converts 3d image to grayscale
    Args:
        im_as_arr (numpy arr): RGB image with shape (D,W,H)
    returns:
        grayscale_im (numpy_arr): Grayscale image with shape (1,W,D)
    """
    grayscale_im = np.sum(np.abs(im_as_arr), axis=0)
    im_max = np.percentile(grayscale_im, 99)
    im_min = np.min(grayscale_im)
    grayscale_im = (np.clip((grayscale_im - im_min) / (im_max - im_min), 0, 1))
    grayscale_im = np.expand_dims(grayscale_im, axis=0)
    return grayscale_im


def save_gradient_images(gradient, file_name):
    """
        Exports the original gradient image
    Args:
        gradient (np arr): Numpy array of the gradient with shape (3, 224, 224)
        file_name (str): File name to be exported
    """
    if not os.path.exists('../results'):
        os.makedirs('../results')
    # Normalize
    gradient = gradient - gradient.min()
    gradient /= gradient.max()
    # Save image
    path_to_file = os.path.join('../results', file_name + '.png')
    save_image(gradient, path_to_file)


def save_class_activation_images(org_img, activation_map, file_name):
    """
        Saves cam activation map and activation map on the original image
    Args:
        org_img (PIL img): Original image
        activation_map (numpy arr): Activation map (grayscale) 0-255
        file_name (str): File name of the exported image
    """
    if not os.path.exists('../results'):
        os.makedirs('../results')
    # Grayscale activation map
    heatmap, heatmap_on_image = apply_colormap_on_image(org_img, activation_map, 'hsv')
    # Save colored heatmap
    path_to_file = os.path.join('../results', file_name+'_Cam_Heatmap.png')
    save_image(heatmap, path_to_file)
    # Save heatmap on iamge
    path_to_file = os.path.join('../results', file_name+'_Cam_On_Image.png')
    save_image(heatmap_on_image, path_to_file)
    # SAve grayscale heatmap
    path_to_file = os.path.join('../results', file_name+'_Cam_Grayscale.png')
    save_image(activation_map, path_to_file)


def apply_colormap_on_image(org_im, activation, colormap_name):
    """
        Apply heatmap on image
    Args:
        org_img (PIL img): Original image
        activation_map (numpy arr): Activation map (grayscale) 0-255
        colormap_name (str): Name of the colormap
    """
    # Get colormap
    color_map = mpl_color_map.get_cmap(colormap_name)
    no_trans_heatmap = color_map(activation)
    # Change alpha channel in colormap to make sure original image is displayed
    heatmap = copy.copy(no_trans_heatmap)
    heatmap[:, :, 3] = 0.4
    heatmap = Image.fromarray((heatmap*255).astype(np.uint8))
    no_trans_heatmap = Image.fromarray((no_trans_heatmap*255).astype(np.uint8))

    # Apply heatmap on image
    heatmap_on_image = Image.new("RGBA", org_im.size)
    heatmap_on_image = Image.alpha_composite(heatmap_on_image, org_im.convert('RGBA'))
    heatmap_on_image = Image.alpha_composite(heatmap_on_image, heatmap)
    return no_trans_heatmap, heatmap_on_image


def apply_heatmap(R, sx, sy):
    """
        Heatmap code stolen from https://git.tu-berlin.de/gmontavon/lrp-tutorial
        This is (so far) only used for LRP
    """
    b = 10*((np.abs(R)**3.0).mean()**(1.0/3))
    my_cmap = plt.cm.seismic(np.arange(plt.cm.seismic.N))
    my_cmap[:, 0:3] *= 0.85
    my_cmap = ListedColormap(my_cmap)
    plt.figure(figsize=(sx, sy))
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.axis('off')
    heatmap = plt.imshow(R, cmap=my_cmap, vmin=-b, vmax=b, interpolation='nearest')
    return heatmap
    # plt.show()


def format_np_output(np_arr):
    """
        This is a (kind of) bandaid fix to streamline saving procedure.
        It converts all the outputs to the same format which is 3xWxH
        with using sucecssive if clauses.
    Args:
        im_as_arr (Numpy array): Matrix of shape 1xWxH or WxH or 3xWxH
    """
    # Phase/Case 1: The np arr only has 2 dimensions
    # Result: Add a dimension at the beginning
    if len(np_arr.shape) == 2:
        np_arr = np.expand_dims(np_arr, axis=0)
    # Phase/Case 2: Np arr has only 1 channel (assuming first dim is channel)
    # Result: Repeat first channel and convert 1xWxH to 3xWxH
    if np_arr.shape[0] == 1:
        np_arr = np.repeat(np_arr, 3, axis=0)
    # Phase/Case 3: Np arr is of shape 3xWxH
    # Result: Convert it to WxHx3 in order to make it saveable by PIL
    if np_arr.shape[0] == 3:
        np_arr = np_arr.transpose(1, 2, 0)
    # Phase/Case 4: NP arr is normalized between 0-1
    # Result: Multiply with 255 and change type to make it saveable by PIL
    if np.max(np_arr) <= 1:
        np_arr = (np_arr*255).astype(np.uint8)
    return np_arr


def save_image(im, path):
    """
        Saves a numpy matrix or PIL image as an image
    Args:
        im_as_arr (Numpy array): Matrix of shape DxWxH
        path (str): Path to the image
    """
    if isinstance(im, (np.ndarray, np.generic)):
        im = format_np_output(im)
        im = Image.fromarray(im)
    im.save(path)


def preprocess_image(pil_im, resize_im=True):
    """
        Processes image for CNNs
    Args:
        PIL_img (PIL_img): PIL Image or numpy array to process
        resize_im (bool): Resize to 224 or not
    returns:
        im_as_var (torch variable): Variable that contains processed float tensor
    """
    # Mean and std list for channels (Imagenet)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    # Ensure or transform incoming image to PIL image
    if type(pil_im) != Image.Image:
        try:
            pil_im = Image.fromarray(pil_im)
        except Exception as e:
            print("could not transform PIL_img to a PIL Image object. Please check input.")

    # Resize image
    if resize_im:
        pil_im = pil_im.resize((224, 224), Image.ANTIALIAS)

    im_as_arr = np.float32(pil_im)
    im_as_arr = im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
        im_as_arr[channel] -= mean[channel]
        im_as_arr[channel] /= std[channel]
    # Convert to float tensor
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    im_as_ten.unsqueeze_(0)
    # Convert to Pytorch variable
    im_as_var = Variable(im_as_ten, requires_grad=True)
    return im_as_var


def recreate_image(im_as_var):
    """
        Recreates images from a torch variable, sort of reverse preprocessing
    Args:
        im_as_var (torch variable): Image to recreate
    returns:
        recreated_im (numpy arr): Recreated image in array
    """
    reverse_mean = [-0.485, -0.456, -0.406]
    reverse_std = [1/0.229, 1/0.224, 1/0.225]
    recreated_im = copy.copy(im_as_var.data.numpy()[0])
    for c in range(3):
        recreated_im[c] /= reverse_std[c]
        recreated_im[c] -= reverse_mean[c]
    recreated_im[recreated_im > 1] = 1
    recreated_im[recreated_im < 0] = 0
    recreated_im = np.round(recreated_im * 255)

    recreated_im = np.uint8(recreated_im).transpose(1, 2, 0)
    return recreated_im


def get_positive_negative_saliency(gradient):
    """
        Generates positive and negative saliency maps based on the gradient
    Args:
        gradient (numpy arr): Gradient of the operation to visualize
    returns:
        pos_saliency ( )
    """
    pos_saliency = (np.maximum(0, gradient) / gradient.max())
    neg_saliency = (np.maximum(0, -gradient) / -gradient.min())
    return pos_saliency, neg_saliency


def get_example_params(example_index):
    """
        Gets used variables for almost all visualizations, like the image, model etc.
    Args:
        example_index (int): Image id to use from examples
    returns:
        original_image (numpy arr): Original image read from the file
        prep_img (numpy_arr): Processed image
        target_class (int): Target class for the image
        file_name_to_export (string): File name to export the visualizations
        pretrained_model(Pytorch model): Model to use for the operations
    """

    
    # Pick one of the examples
    example_list = (('input_images/805.jpeg', 805),
                    ('input_images/image_254.jpeg', 245),
                    ('input_images/ostrich_hammer_vgg16.png', 587),
                    ('input_images/ostrich_labcoat_vgg16.png', 617),
                    ('input_images/ostrich_acornsquash_vgg16.png', 941),
                    ('input_images/ostrich_goldfinch_vgg16.png', 11),
                    ('input_images/corn_hammer_vgg16.png', 587),
                    ('input_images/corn_labcoat_vgg16.png', 617),
                    ('input_images/corn_acornsquash_vgg16.png', 941),
                    ('input_images/corn_goldfinch_vgg16.png', 11))
    img_path = example_list[example_index][0]
    target_class = example_list[example_index][1]
    file_name_to_export = img_path[img_path.rfind('/')+1:img_path.rfind('.')]
    # Read image
    original_image = Image.open(img_path).convert('RGB')
    # Process image
    prep_img = preprocess_image(original_image)
    # Define model
    pretrained_model = models.alexnet(pretrained=True)
    return (original_image,
            prep_img,
            target_class,
            file_name_to_export,
            pretrained_model)

In [ ]:
#@title create class specific adversarial image
# Create adversarial image

import os
import numpy as np

import torch
from torch.optim import SGD
from torchvision import models

class ClassSpecificImageGeneration():
    """
        Produces an image that maximizes a certain class with gradient ascent
    """
    def __init__(self, model, starting_image, target_class):
        self.mean = [-0.485, -0.456, -0.406]
        self.std = [1/0.229, 1/0.224, 1/0.225]
        self.model = model
        self.model.eval()
        self.target_class = target_class
        # Generate a random image
        self.created_image = recreate_image(starting_image)
        # Create the folder to export images if not exists
        if not os.path.exists('../generated/class_'+str(self.target_class)):
            os.makedirs('../generated/class_'+str(self.target_class))

    def generate(self, iterations=150):
        """Generates class specific image
        Keyword Arguments:
            iterations {int} -- Total iterations for gradient ascent (default: {150})
        Returns:
            np.ndarray -- Final maximally activated class image
        """
        initial_learning_rate = 6
        for i in range(1, iterations):
            # Process image and return variable
            self.processed_image = preprocess_image(self.created_image, False)

            # Define optimizer for the image
            optimizer = SGD([self.processed_image], lr=initial_learning_rate)
            # Forward
            output = self.model(self.processed_image)
            # Target specific class
            class_loss = -output[0, self.target_class]

            if i % 10 == 0 or i == iterations-1:
                print('Iteration:', str(i), 'Loss',
                      "{0:.2f}".format(class_loss.data.numpy()))
            # Zero grads
            self.model.zero_grad()
            # Backward
            class_loss.backward()
            # Update image
            optimizer.step()
            # Recreate image
            self.created_image = recreate_image(self.processed_image)
            # if i % 10 == 0 or i == iterations-1:
            #     # Save image
            #     im_path = '../generated/class_'+str(self.target_class)+'/c_'+str(self.target_class)+'_'+'iter_'+str(i)+'.png'
            #     save_image(self.created_image, im_path)

        return self.processed_image

Next Steps (discussed between Sohum and Mohit):
1. Pick a pretrainted model and dataset
2. Find a correctly classified input
3. Generate a perturbed input [example](https://medium.com/@ml.at.berkeley/tricking-neural-networks-create-your-own-adversarial-examples-a61eb7620fd8)
4. Provide visualizations of these inputs as they go through the pretrained model
5. Report findings about where they diverge, where the outputs look very different
6. Organize results in a way that communicates this understanding

In [ ]:
target_example = 0  # Corn
(corn_image, corn_tensor, corn_class, file_name_to_export, pretrained_model) = get_example_params(target_example)

#target_example = 1 # Ostrich
#(ostrich_image, ostrich_tensor, ostrich_class, file_name_to_export, pretrained_model) = get_example_params(target_example)


In [ ]:
with torch.no_grad():
    output = pretrained_model(corn_tensor)
torch.argmax(output[0]).item()

805

In [ ]:
adversarial_classes = [(932, "pretzel"), (112, "conch"), (526, "desk"), (499, "cleaver")]

for adversarial_class, label in adversarial_classes:

# adversarial_class = 56
# label = "kingsnake"

  csig = ClassSpecificImageGeneration(pretrained_model, corn_tensor, adversarial_class)
  output_tensor = csig.generate(iterations = 5)
  with torch.no_grad():
    classification = pretrained_model(output_tensor)

  classification = torch.argmax(classification[0]).item()
  print(classification)

  output_np = recreate_image(output_tensor)
  output_image = Image.fromarray(output_np)
  save_image(output_image, "input_images/corn_" + label + "_alexnet.png")

!zip -r adversarial_images.zip input_images/

Iteration: 4 Loss -20.54
932
Iteration: 4 Loss -32.36
112
Iteration: 4 Loss -17.68
526
Iteration: 4 Loss -17.02
499
updating: input_images/ (stored 0%)
updating: input_images/805.jpeg (stored 0%)
updating: input_images/image_254.jpeg (deflated 1%)
updating: input_images/corn_parachute_alexnet.png (deflated 0%)
updating: input_images/corn_analog_clock_alexnet.png (deflated 0%)
updating: input_images/corn_submarine_alexnet.png (deflated 0%)
updating: input_images/.ipynb_checkpoints/ (stored 0%)
updating: input_images/corn_sandal_alexnet.png (deflated 0%)
updating: input_images/corn_bulletproof_vest_alexnet.png (deflated 0%)
updating: input_images/corn_pretzel_alexnet.png (deflated 0%)
updating: input_images/corn_conch_alexnet.png (deflated 0%)
updating: input_images/corn_flute_alexnet.png (deflated 0%)
  adding: input_images/corn_cleaver_alexnet.png (deflated 0%)
  adding: input_images/corn_desk_alexnet.png (deflated 0%)


In [ ]:
for adversarial_class, label in adversarial_classes:

  csig = ClassSpecificImageGeneration(pretrained_model, ostrich_tensor, adversarial_class)
  output_tensor = csig.generate(iterations = 5)
  with torch.no_grad():
    classification = pretrained_model(output_tensor)

  classification = torch.argmax(classification[0]).item()
  print(classification)

  output_np = recreate_image(output_tensor)
  output_image = Image.fromarray(output_np)
  save_image(output_image, "input_images/ostrich_" + label + "_alexnet.png")

!zip -r adversarial_images.zip input_images/

Iteration: 4 Loss -19.80
695
Iteration: 4 Loss -25.87
617
Iteration: 4 Loss -23.35
941
Iteration: 4 Loss -11.84
11
updating: input_images/ (stored 0%)
updating: input_images/corn_labcoat_alexnet.png (deflated 0%)
updating: input_images/805.jpeg (stored 0%)
updating: input_images/image_254.jpeg (deflated 1%)
updating: input_images/corn_goldfinch_alexnet.png (deflated 0%)
updating: input_images/corn_hammer_alexnet.png (deflated 0%)
updating: input_images/corn_acornsquash_alexnet.png (deflated 0%)
  adding: input_images/ostrich_acornsquash_alexnet.png (deflated 0%)
  adding: input_images/ostrich_labcoat_alexnet.png (deflated 0%)
  adding: input_images/ostrich_goldfinch_alexnet.png (deflated 0%)
  adding: input_images/ostrich_hammer_alexnet.png (deflated 0%)


## Saliency Maps


In [ ]:
import theano
import theano.tensor as T

def compile_saliency_function(net):
    """
    Compiles a function to compute the saliency maps and predicted classes
    for a given minibatch of input images.
    """
    inp = net['input'].input_var
    outp = lasagne.layers.get_output(net['fc8'], deterministic=True)
    max_outp = T.max(outp, axis=1)
    saliency = theano.grad(max_outp.sum(), wrt=inp)
    max_class = T.argmax(outp, axis=1)
    return theano.function([inp], [saliency, max_class])

def show_images(img_original, saliency, max_class, title):
    # get out the first map and class from the mini-batch
    saliency = saliency[0]
    max_class = max_class[0]
    # convert saliency from BGR to RGB, and from c01 to 01c
    saliency = saliency[::-1].transpose(1, 2, 0)
    # plot the original image and the three saliency map variants
    plt.figure(figsize=(10, 10), facecolor='w')
    plt.suptitle("Class: " + classes[max_class] + ". Saliency: " + title)
    plt.subplot(2, 2, 1)
    plt.title('input')
    plt.imshow(img_original)
    plt.subplot(2, 2, 2)
    plt.title('abs. saliency')
    plt.imshow(np.abs(saliency).max(axis=-1), cmap='gray')
    plt.subplot(2, 2, 3)
    plt.title('pos. saliency')
    plt.imshow((np.maximum(0, saliency) / saliency.max()))
    plt.subplot(2, 2, 4)
    plt.title('neg. saliency')
    plt.imshow((np.maximum(0, -saliency) / -saliency.min()))
    plt.show()

import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'jpeg'
%matplotlib inline
import urllib
import io
import skimage.transform

def prepare_image(url):
    ext = url.rsplit('.', 1)[1]
    img = plt.imread(io.BytesIO(urllib.urlopen(url).read()), ext)
    # Resize so smallest dim = 256, preserving aspect ratio
    h, w, _ = img.shape
    if h < w:
        img = skimage.transform.resize(img, (256, w*256/h), preserve_range=True)
    else:
        img = skimage.transform.resize(img, (h*256/w, 256), preserve_range=True)
    # Central crop to 224x224
    h, w, _ = img.shape
    img = img[h//2-112:h//2+112, w//2-112:w//2+112]
    # Remember this, it's a single RGB image suitable for plt.imshow()
    img_original = img.astype('uint8')
    # Shuffle axes from 01c to c01
    img = img.transpose(2, 0, 1)
    # Convert from RGB to BGR
    img = img[::-1]
    # Subtract mean pixel value
    img = img - mean_pixel[:, np.newaxis, np.newaxis]
    # Return the original and the prepared image (as a batch of a single item)
    return img_original, lasagne.utils.floatX(img[np.newaxis])

ModuleNotFoundError: ignored

#VGG

In [ ]:
pretrained_model = models.vgg16(pretrained=True)
pretrained_model

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
adversarial_classes = [(587, "hammer"), (617, "labcoat"), (941, "acornsquash"), (11, "goldfinch")]

for adversarial_class, label in adversarial_classes:

# adversarial_class = 56
# label = "kingsnake"

  csig = ClassSpecificImageGeneration(pretrained_model, corn_tensor, adversarial_class)
  output_tensor = csig.generate(iterations = 5)
  with torch.no_grad():
    classification = pretrained_model(output_tensor)

  classification = torch.argmax(classification[0]).item()
  print(classification)

  output_np = recreate_image(output_tensor)
  output_image = Image.fromarray(output_np)
  save_image(output_image, "input_images/corn_" + label + "_vgg16.png")

!zip -r adversarial_images.zip input_images/

Iteration: 4 Loss -12.21
462
Iteration: 4 Loss -10.13
998
Iteration: 4 Loss -20.27
941
Iteration: 4 Loss -15.26
318
updating: input_images/ (stored 0%)
updating: input_images/corn.JPEG (deflated 0%)
updating: input_images/ostrich.JPEG (deflated 0%)
updating: input_images/.ipynb_checkpoints/ (stored 0%)
  adding: input_images/corn_labcoat_vgg16.png (deflated 0%)
  adding: input_images/corn_goldfinch_vgg16.png (deflated 0%)
  adding: input_images/corn_hammer_vgg16.png (deflated 0%)
  adding: input_images/corn_acornsquash_vgg16.png (deflated 0%)


In [ ]:
for adversarial_class, label in adversarial_classes:

  csig = ClassSpecificImageGeneration(pretrained_model, ostrich_tensor, adversarial_class)
  output_tensor = csig.generate(iterations = 5)
  with torch.no_grad():
    classification = pretrained_model(output_tensor)

  classification = torch.argmax(classification[0]).item()
  print(classification)

  output_np = recreate_image(output_tensor)
  output_image = Image.fromarray(output_np)
  save_image(output_image, "input_images/ostrich_" + label + "_vgg16.png")

!zip -r adversarial_images.zip input_images/

Iteration: 4 Loss -32.94
587
Iteration: 4 Loss -10.58
617
Iteration: 4 Loss -26.32
941
Iteration: 4 Loss -14.66
84
updating: input_images/ (stored 0%)
updating: input_images/corn.JPEG (deflated 0%)
updating: input_images/ostrich.JPEG (deflated 0%)
updating: input_images/.ipynb_checkpoints/ (stored 0%)
updating: input_images/corn_labcoat_vgg16.png (deflated 0%)
updating: input_images/corn_goldfinch_vgg16.png (deflated 0%)
updating: input_images/corn_hammer_vgg16.png (deflated 0%)
updating: input_images/corn_acornsquash_vgg16.png (deflated 0%)
  adding: input_images/ostrich_acornsquash_vgg16.png (deflated 0%)
  adding: input_images/ostrich_goldfinch_vgg16.png (deflated 0%)
  adding: input_images/ostrich_hammer_vgg16.png (deflated 0%)
  adding: input_images/ostrich_labcoat_vgg16.png (deflated 0%)


## Saliency Maps

In [ ]:
url = 'http://farm5.static.flickr.com/4064/4334173592_145856d89b.jpg'
img_original, img = prepare_image(url)

saliency_fn = compile_saliency_function(pretrained_model)
saliency, max_class = saliency_fn(img)
show_images(img_original, saliency, max_class, "default gradient")

In [ ]:
pip install tf-keras-vis


In [ ]:
#dependencies
import shutil
import os
import random
import glob
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


#Unecessary since we already have our model trained

"""
#extracting the images
filename = "train.zip"
shutil.unpack_archive(filename, "images")

#storing cat and dog files in lists
cat_files = glob.glob("images/train/cat*.jpg")
dog_files = glob.glob("images/train/dog*.jpg")

#make training and test dir
os.mkdir("images/test")
os.mkdir("images/train/dog")
os.mkdir("images/train/cat")
os.mkdir("images/test/dog")
os.mkdir("images/test/cat")

#moving training data and testing data to the appropriate folders
training_dir = "images/train"
testing_dir = "images/test"

training_dog = "images/train/dog"
training_cat = "images/train/cat"

testing_dog = "images/test/dog"
testing_cat = "images/test/cat"

#only using 1000 and 100 per class out of 12500 images for training and testing, respectively  
train_dog_files = random.sample(dog_files, 1000)
train_cat_files = random.sample(cat_files, 1000)
test_dog_files = [file for file in random.sample(dog_files, 100) if file not in train_dog_files]
test_cat_files = [file for file in random.sample(cat_files, 100) if file not in train_cat_files]

for file in train_dog_files:
    shutil.move(file, "images/train/dog")

for file in train_cat_files:
    shutil.move(file, "images/train/cat")

for file in test_dog_files:
    shutil.move(file, "images/test/dog")

for file in test_cat_files:
    shutil.move(file, "images/test/cat")

print(f'total training dog images: {len(os.listdir(training_dog))}')
print(f'total training cat images: {len(os.listdir(training_cat))}')
print(f'total validation dog images: {len(os.listdir(testing_dog))}')
print(f'total validation cat images: {len(os.listdir(testing_cat))}')



# All images will be rescaled by 1./255 with some augmentation applied to training images
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        training_dir,  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=128,
        # Since you use binary_crossentropy loss, you need binary labels
        class_mode='binary')

# Flow validation images in batches of 128 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        testing_dir,  # This is the source directory for validation images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=32,
        # Since you use binary_crossentropy loss, you need binary labels
        class_mode='binary')

#model architecture
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. 
    # It will contain a value from 0-1 where 0 for 1 class ('dog') and 1 for the other ('cat')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

#model compilation
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

#model training
history = model.fit(
      train_generator,
      epochs=10,
      verbose=1,
      validation_data = validation_generator)

"""

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils import normalize
from vis.utils import utils

#loading the image and converting it to numpy array
#img = tf.keras.preprocessing.image.load_img('images/test/dog/'+ random.sample(os.listdir(testing_dog), 1)[0],target_size=(300,300))
img = tf.keras.preprocessing.image.load_img('hen.png', target_size=(300,300))

x = img_to_array(img)  # Numpy array with shape (300, 300, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 300, 300, 3)

# swap last layer with linear layer 
layer_idx = utils.find_layer_idx(model, model.layers[-1].name)
model.layers[-1].activation = tf.keras.activations.linear
model = utils.apply_modifications(model)

from tf_keras_vis.utils.scores import CategoricalScore
score = CategoricalScore([0])

#Create Saliency object
saliency = Saliency(model, clone=False)

subplot_args = {
   'nrows': 1,
   'ncols': 1,
   'figsize': (5, 4),
   'subplot_kw': {'xticks': [], 'yticks': []}
}

# Generate saliency map
saliency_map = saliency(score, x, smooth_samples=20, smooth_noise=0.2)
saliency_map = normalize(saliency_map)

f, ax = plt.subplots(**subplot_args)
ax.imshow(saliency_map[0], cmap='Reds')
plt.tight_layout()
plt.show()

In [ ]:
!pip install flashtorch torch==1.5.0 torchvision==0.6.0 -U



In [ ]:
import matplotlib.pyplot as plt
import torchvision.models as models

from flashtorch.utils import apply_transforms, load_image
from flashtorch.saliency import Backprop

image = load_image('hen.png')

plt.imshow(image)
plt.title('Original image')
plt.axis('off');

In [ ]:
model = models.alexnet(pretrained=True)
backprop = Backprop(model)



In [ ]:

img = apply_transforms(image)

target_class = 8

#8 for hen and 56 for the snake


backprop.visualize(img, target_class, guided=True, use_gpu=True)